## Game pattern and bye week constraints

In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np


#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}


W=set(list(range(0,13))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
#set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)


In [2]:
week = 13

In [3]:
# Create an empty model
m= gp.Model(name="NFL2b")
# ADD DECISION VARIABLES HERE
x = m.addVars(T, T, W, vtype = GRB.BINARY, name = "X")

Using license file C:\Users\Q_Ali\gurobi.lic
Academic license - for non-commercial use only


In [4]:
# ADD CONSTRAINTS
# add constraints that The season was limited to 12 weeks
m.addConstrs(sum(sum(x[i,j,k]+x[j,i,k] for k in W) for i in T) == 12 for j in T)
# add constraints that Each team would play once per week
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(0,8))
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(9,13))
# add constraints that All 12 games that a team played would need to be against a different opponent
m.addConstrs(sum(x[i,j,k]+x[j,i,k]for k in W) <= 1 for i in T for j in T)
# add constraints that Each team would play at most six home games (i.e., on their home stadium)
m.addConstrs(sum(sum(x[i,j,k] for k in W) for j in T) <=6 for i in T)
# add constraints to avoid circumstances that i=i
m.addConstrs(x[i,i,k] == 0 for i in T for i in T for k in W)
# add constraints to let binary variables are bound between 0 and 1
m.addConstrs(x[i,j,k] <= 1 for i in T for j in T for k in W)
m.addConstrs(x[i,j,k] >= 0 for i in T for j in T for k in W)

# add gaming pattern constraints for 2a
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) <= 2 for i in T for k in range(week-2))
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) >= 1 for i in T for k in range(week-2))

# add gaming pattern constraints for there are no games on week9
m.addConstrs(x[i,j,8] == 0 for i in T for j in T)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [5]:
# ADD OBJECTIVE FUNCTION
# Objective is to minimize the total travel distance of all teams
obj = sum(E.iloc[i,j]*x[i,j,k] for i in T for j in T for k in W)*2

In [6]:
#optimize the model
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 24480 rows, 7488 columns and 111408 nonzeros
Model fingerprint: 0x15c04b36
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 23484 rows and 864 columns
Presolve time: 0.15s
Presolved: 996 rows, 6624 columns, 52992 nonzeros
Variable types: 0 continuous, 6624 integer (6624 binary)

Root relaxation: objective 1.659080e+05, 4857 iterations, 0.66 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  307          - 165908.000      -     -    2s
     0     0 165908.000    0  482          - 165908.000      -     -    3s
H    0     0                    178224.00000 165908.000  6.91%     -    3s
     0    

In [7]:
# ADD PRINTING
# Print optimal value of the objective function
print('\nValue of objective function: %g' % m.objVal)



Value of objective function: 165908


In [8]:
# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    if v.x !=0:
        print(f'{v.varName}={v.x}, {v.obj}')



Decision variables:
X[0,1,5]=1.0, 490.0
X[0,6,9]=1.0, 940.0
X[0,13,11]=1.0, 922.0
X[0,16,6]=1.0, 1068.0
X[0,17,3]=1.0, 692.0
X[0,23,2]=1.0, 1586.0
X[1,6,12]=1.0, 1430.0
X[1,10,9]=1.0, 814.0
X[1,15,6]=1.0, 2076.0
X[1,17,1]=1.0, 766.0
X[1,18,11]=1.0, 1434.0
X[1,19,3]=1.0, 1648.0
X[2,3,7]=1.0, 568.0
X[2,4,2]=1.0, 414.0
X[2,12,12]=1.0, 1056.0
X[2,14,3]=1.0, 688.0
X[2,15,5]=1.0, 2164.0
X[2,23,10]=1.0, 1934.0
X[3,5,11]=1.0, 1382.0
X[3,7,1]=1.0, 1226.0
X[3,14,9]=1.0, 340.0
X[3,19,2]=1.0, 1412.0
X[3,20,6]=1.0, 1226.0
X[3,21,4]=1.0, 572.0
X[4,3,3]=1.0, 978.0
X[4,5,0]=1.0, 558.0
X[4,12,6]=1.0, 1468.0
X[4,13,9]=1.0, 1014.0
X[4,22,12]=1.0, 1366.0
X[4,23,1]=1.0, 2346.0
X[5,0,1]=1.0, 2226.0
X[5,2,6]=1.0, 818.0
X[5,6,3]=1.0, 2398.0
X[5,12,9]=1.0, 1870.0
X[5,16,12]=1.0, 1182.0
X[5,22,4]=1.0, 1758.0
X[6,2,4]=1.0, 1850.0
X[6,4,5]=1.0, 2260.0
X[6,9,10]=1.0, 1314.0
X[6,15,0]=1.0, 696.0
X[6,18,2]=1.0, 1702.0
X[6,23,7]=1.0, 1010.0
X[7,1,0]=1.0, 1250.0
X[7,9,3]=1.0, 2262.0
X[7,11,6]=1.0, 378.0
X[7,18,5]=1.0